# Optimisation hyperparamètres des modèles avec le features engineering

On va optimiser les hyperparamètres des modèles prometteurs que nous avons identifiés dans le notebook précédent. 

On va utiliser des méthodes de recherche d'hyperparamètres par validation croisée pour éviter le surapprentissage.

On va utiliser les méthodes bayésiennes pour optimiser les hyperparamètres des modèles de manière intelligente.

In [7]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('train.csv')

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On divise les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# On normalise les données continues 
from sklearn.preprocessing import StandardScaler

continuous_columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                      'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                      'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
scaler = StandardScaler()

# On applique le scaler uniquement sur les colonnes continues
#X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
#X_test[continuous_columns] = scaler.transform(X_test[continuous_columns])

In [9]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [10]:
# Dictionnaire des types de sols avec leurs codes ELU
soil_type_to_elu = {
    1: "2702",  2: "2703",  3: "2704",  4: "2705",  5: "2706",  6: "2717",
    7: "3501",  8: "3502",  9: "4201", 10: "4703", 11: "4704", 12: "4744",
    13: "4758", 14: "5101", 15: "5151", 16: "6101", 17: "6102", 18: "6731",
    19: "7101", 20: "7102", 21: "7103", 22: "7201", 23: "7202", 24: "7700",
    25: "7701", 26: "7702", 27: "7709", 28: "7710", 29: "7745", 30: "7746",
    31: "7755", 32: "7756", 33: "7757", 34: "7790", 35: "8703", 36: "8707",
    37: "8708", 38: "8771", 39: "8772", 40: "8776"
}

# Fonction pour extraire les composantes du code ELU
def extract_elu_components(elu_code):
    climatic_zone = int(elu_code[0])  # Premier chiffre pour la zone climatique
    geologic_zone = int(elu_code[1])  # Deuxième chiffre pour la zone géologique
    return climatic_zone, geologic_zone

# Fonction pour ajouter les colonnes climatic_zone et geologic_zone
def add_climatic_geologic_zones(df, soil_type_to_elu):
    # Faire une copie explicite du DataFrame pour éviter les effets de référence
    df = df.copy()
    
    # Initialiser des listes pour stocker les valeurs des zones climatiques et géologiques
    climatic_zones = []
    geologic_zones = []

    # Parcourir chaque ligne pour identifier le type de sol
    for i, row in df.iterrows():
        # Filtrer les colonnes `Soil_TypeX` pour cette ligne
        soil_type_columns = [col for col in df.columns if 'Soil_Type' in col]
        soil_type_values = row[soil_type_columns]
        
        # Vérifier si une seule colonne `Soil_TypeX` a la valeur 1
        if soil_type_values.sum() == 1:
            # Identifier le type de sol en trouvant la colonne avec la valeur 1
            soil_type = int(soil_type_values.idxmax().split('Soil_Type')[1])
            
            # Récupérer le code ELU pour le type de sol
            elu_code = soil_type_to_elu.get(soil_type)
            
            # Vérification supplémentaire : s'assurer que le code ELU existe
            if elu_code:
                # Extraire la zone climatique et géologique à partir du code ELU
                climatic_zone, geologic_zone = extract_elu_components(elu_code)
            else:
                # Si le code ELU est manquant, utiliser -1
                climatic_zone, geologic_zone = -1, -1
        else:
            # Si aucune colonne ou plusieurs colonnes sont égales à 1, utiliser -1
            climatic_zone, geologic_zone = -1, -1

        # Ajouter les valeurs aux listes
        climatic_zones.append(climatic_zone)
        geologic_zones.append(geologic_zone)

    # Vérifier que les longueurs des listes correspondent au nombre de lignes dans df
    assert len(climatic_zones) == len(df), "Erreur : La longueur des listes ne correspond pas au nombre de lignes du DataFrame"

    # Ajouter les nouvelles colonnes au DataFrame en utilisant `assign`
    df = df.assign(
        climatic_zone=pd.Series(climatic_zones, index=df.index, dtype="float"),
        geologic_zone=pd.Series(geologic_zones, index=df.index, dtype="float")
    )
    
    return df

In [57]:
X_train_soil = add_climatic_geologic_zones(X_train, soil_type_to_elu)

# Vérifier le nombre de NaN dans les colonnes `climatic_zone` et `geologic_zone`
nan_counts = X_train_soil[['climatic_zone', 'geologic_zone']].isna().sum()
print("Nombre de NaN dans chaque colonne :")
print(nan_counts)

X_test_soil = add_climatic_geologic_zones(X_test, soil_type_to_elu)

Nombre de NaN dans chaque colonne :
climatic_zone    0
geologic_zone    0
dtype: int64


## Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

# On définit l'espace de recherche pour Random Forest
space_rf = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [5, 10, 15, 20, None]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'max_features': hp.choice('max_features', ['sqrt', 'log2'])  # Correction : 'auto' remplacé par 'sqrt'
}

# On crée la fonction objectif pour Random Forest
def objective_rf(params):
    # On initialise le modèle avec les paramètres proposés
    model = RandomForestClassifier(**params, random_state=42)
    # On effectue une validation croisée pour évaluer la performance
    score = cross_val_score(model, X_train_soil, y_train, cv=8, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance Hyperopt pour optimiser les hyperparamètres de Random Forest
trials_rf = Trials()
best_params_rf = fmin(fn=objective_rf, space=space_rf, algo=tpe.suggest, max_evals=50, trials=trials_rf)
print("Meilleurs paramètres Random Forest :", best_params_rf)

100%|██████████| 50/50 [17:24<00:00, 20.88s/trial, best loss: -0.8731103552532125]
Meilleurs paramètres Random Forest : {'max_depth': 4, 'max_features': 0, 'min_samples_leaf': 0, 'min_samples_split': 0, 'n_estimators': 4}


On teste ensuite le modèle avec les hyperparamètres optimisés sur les données de test.

In [58]:
# Traduction des indices en valeurs concrètes
best_rf_params = {
    'max_depth': [5, 10, 15, 20, None][4],           
    'max_features': ['sqrt', 'log2'][0],             
    'min_samples_leaf': [1, 2, 4][0],                
    'min_samples_split': [2, 5, 10][0],              
    'n_estimators': [100, 200, 300, 400, 500][4]     
}

# Initialiser le modèle avec les valeurs concrètes
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
rf_model.fit(X_train_soil, y_train)

# Prédire sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test_soil)

# Évaluer les performances du modèle
from sklearn.metrics import accuracy_score, classification_report
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Précision Random Forest :", accuracy_rf)
print("Rapport de classification Random Forest :\n", classification_report(y_test, y_pred_rf))

Précision Random Forest : 0.8761022927689595
Rapport de classification Random Forest :
               precision    recall  f1-score   support

           1       0.79      0.80      0.80       648
           2       0.81      0.71      0.76       648
           3       0.84      0.88      0.86       648
           4       0.95      0.97      0.96       648
           5       0.92      0.95      0.93       648
           6       0.87      0.87      0.87       648
           7       0.94      0.95      0.94       648

    accuracy                           0.88      4536
   macro avg       0.87      0.88      0.87      4536
weighted avg       0.87      0.88      0.87      4536



## XGBoost

In [17]:
import xgboost as xgb

y_train_adj = y_train - 1
y_test_adj = y_test - 1

# On définit l'espace de recherche pour XGBoost
space_xgb = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [3, 6, 9, 12]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5)
}

# On crée la fonction objectif pour XGBoost
def objective_xgb(params):
    # On initialise XGBoost avec les paramètres proposés
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, **params, random_state=42)
    # On utilise une validation croisée pour calculer la précision
    score = cross_val_score(model, X_train_soil, y_train_adj, cv=8, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance l'optimisation avec Hyperopt pour XGBoost
trials_xgb = Trials()
best_params_xgb = fmin(fn=objective_xgb, space=space_xgb, algo=tpe.suggest, max_evals=50, trials=trials_xgb)
print("Meilleurs paramètres XGBoost :", best_params_xgb)

100%|██████████| 50/50 [1:27:23<00:00, 104.86s/trial, best loss: -0.8877551020408163]  
Meilleurs paramètres XGBoost : {'colsample_bytree': 0.5648136746434348, 'gamma': 0.015587699613612005, 'learning_rate': 0.11668163771105296, 'max_depth': 3, 'n_estimators': 3, 'subsample': 0.9610642479341145}


On teste ensuite le modèle avec les hyperparamètres optimisés sur les données de test.

In [59]:
# On utilise les meilleurs hyperparamètres trouvés par Hyperopt
best_xgb_params = {
    'colsample_bytree': 0.5648136746434348,
    'gamma': 0.015587699613612005,
    'learning_rate': 0.11668163771105296,
    'max_depth': [3, 6, 9, 12][3],
    'n_estimators': [100, 200, 300, 400, 500][3],  
    'subsample': 0.9610642479341145
}

# Initialiser le modèle XGBoost avec les paramètres optimaux
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, **best_xgb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
xgb_model.fit(X_train_soil, y_train_adj)

# Prédire les classes sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_soil)

# Évaluer les performances du modèle
accuracy_xgb = accuracy_score(y_test_adj, y_pred_xgb)
print("Précision du modèle XGBoost :", accuracy_xgb)
print("Rapport de classification XGBoost :\n", classification_report(y_test_adj, y_pred_xgb))

Précision du modèle XGBoost : 0.8910934744268078
Rapport de classification XGBoost :
               precision    recall  f1-score   support

           0       0.81      0.79      0.80       648
           1       0.81      0.75      0.78       648
           2       0.89      0.90      0.89       648
           3       0.96      0.98      0.97       648
           4       0.92      0.96      0.94       648
           5       0.90      0.91      0.91       648
           6       0.94      0.95      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536



## LightGBM

In [ ]:
import lightgbm as lgb

# On définit l'espace de recherche pour LightGBM
space_lgb = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [5, 10, 15, 20, -1]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'num_leaves': hp.choice('num_leaves', [31, 50, 70, 100]),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# On crée la fonction objectif pour LightGBM
def objective_lgb(params):
    # On initialise LightGBM avec les paramètres proposés
    model = lgb.LGBMClassifier(objective='multiclass', num_class=7,**params, verbosity = -1,random_state=42)
    # On utilise la validation croisée pour évaluer la précision
    score = cross_val_score(model, X_train_soil, y_train, cv=8, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance l'optimisation avec Hyperopt pour LightGBM
trials_lgb = Trials()
best_params_lgb = fmin(fn=objective_lgb, space=space_lgb, algo=tpe.suggest, max_evals=50, trials=trials_lgb)
print("Meilleurs paramètres LightGBM :", best_params_lgb)

100%|██████████| 50/50 [1:03:07<00:00, 75.74s/trial, best loss: -0.8921012849584278]
Meilleurs paramètres LightGBM : {'colsample_bytree': 0.6006084259672398, 'learning_rate': 0.037177160869918895, 'max_depth': 3, 'n_estimators': 3, 'num_leaves': 3, 'subsample': 0.811448630394428}


On teste ensuite le modèle avec les hyperparamètres optimisés sur les données de test.

In [60]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# Meilleurs paramètres de LightGBM (après avoir traduit les indices en valeurs)
best_lgb_params = {
    'colsample_bytree': 0.6006084259672398,
    'learning_rate': 0.037177160869918895,
    'max_depth': [5, 10, 15, 20, -1][3],
    'n_estimators': [100, 200, 300, 400, 500][3],  
    'num_leaves': [31, 50, 70, 100][3],             
    'subsample': 0.811448630394428,
    'verbosity': -1  # Désactive les avertissements pendant l'entraînement
}

# Initialiser le modèle LightGBM avec les paramètres optimisés
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=7, **best_lgb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
lgb_model.fit(X_train_soil, y_train)

# Prédire les classes sur l'ensemble de test
y_pred_lgb = lgb_model.predict(X_test_soil)

# Évaluer les performances du modèle
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("Précision du modèle LightGBM :", accuracy_lgb)
print("Rapport de classification LightGBM :\n", classification_report(y_test, y_pred_lgb))

Précision du modèle LightGBM : 0.8919753086419753
Rapport de classification LightGBM :
               precision    recall  f1-score   support

           1       0.81      0.79      0.80       648
           2       0.81      0.75      0.77       648
           3       0.89      0.90      0.90       648
           4       0.96      0.98      0.97       648
           5       0.93      0.96      0.94       648
           6       0.90      0.91      0.91       648
           7       0.94      0.95      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536



## Stacking

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# Définir les modèles de base avec les meilleurs hyperparamètres
random_forest = RandomForestClassifier(
    n_estimators=500,  
    max_depth=None,      
    random_state=42,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
)

xgboost = xgb.XGBClassifier(
    colsample_bytree=0.5648136746434348,
    gamma=0.015587699613612005,
    learning_rate=0.11668163771105296,
    max_depth=12,
    n_estimators=400,
    subsample=0.9610642479341145,
    objective='multi:softmax',
    num_class=7,
    random_state=42
)

lightgbm = lgb.LGBMClassifier(
    colsample_bytree=0.6006084259672398,
    learning_rate=0.037177160869918895,
    max_depth=20,
    n_estimators=400,
    num_leaves=100,
    subsample=0.811448630394428,
    objective='multiclass',
    num_class=7,
    random_state=42,
    verbosity=-1
)

# Définir le méta-modèle (un modèle simple comme la régression logistique)
meta_model = LogisticRegression(max_iter=1000)

# Créer le StackingClassifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', random_forest),
        ('xgb', xgboost),
        ('lgbm', lightgbm)
    ],
    final_estimator=meta_model,
    cv=10  # Utiliser une validation croisée pour le stacking
)

In [72]:
# Entraîner le modèle de stacking
stacking_model.fit(X_train_soil, y_train_adj)

# Prédire sur l'ensemble de test
X_test_soil = add_climatic_geologic_zones(X_test, soil_type_to_elu)
y_pred_stack = stacking_model.predict(X_test_soil)

# Évaluer les performances du modèle de stacking
accuracy_stack = accuracy_score(y_test_adj, y_pred_stack)
print("Précision du modèle de stacking :", accuracy_stack)
print("Rapport de classification du modèle de stacking :\n", classification_report(y_test_adj, y_pred_stack))

Précision du modèle de stacking : 0.8893298059964727
Rapport de classification du modèle de stacking :
               precision    recall  f1-score   support

           0       0.80      0.80      0.80       648
           1       0.79      0.75      0.77       648
           2       0.89      0.91      0.90       648
           3       0.96      0.98      0.97       648
           4       0.94      0.94      0.94       648
           5       0.90      0.91      0.90       648
           6       0.95      0.94      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536



## Test complet

In [68]:
X_soil = add_climatic_geologic_zones(X, soil_type_to_elu)
y_adj = y - 1


X_test_full = pd.read_csv('test-full.csv')
X_test_full_soil = add_climatic_geologic_zones(X_test_full, soil_type_to_elu)

In [73]:
# On entraîne le modèle de stacking
stacking_model.fit(X_soil, y_adj)

# On prédit sur l'ensemble de test
y_pred_stack = stacking_model.predict(X_test_full_soil)

y_pred_stack = y_pred_stack + 1

In [74]:
# On crée un DataFrame avec les Id et les Cover_Type prédits
submission_df = pd.DataFrame({
    'Id': X_test_full['Id'],
    'Cover_Type': y_pred_stack
})

# On sauvegarde le DataFrame en fichier CSV
submission_df.to_csv('soumissions/submission_soil_opt10.csv', index=False)